In [ ]:
device = "cpu"
# device = "cuda"
import time,os,copy,torch,pytorch_spiking,torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
from tqdm import tqdm
import pickle
# Pennylane
import pennylane as qml
from pennylane import numpy as np
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Plotting
import matplotlib.pyplot as plt

# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "16"
dataset = "FASHIONMNIST"

In [ ]:
spikeaware_model = torch.load("spikeaware_model2-Copy1")
display(spikeaware_model)

In [ ]:
n_qubits = 6             # Number of qubits
nqubits=n_qubits
q_depth = 2            # Depth of the quantum circuit (number of variational layers)

q_delta = 0.01              # Initial spread of random quantum weights

tensor_length = n_qubits*(n_qubits-1)*q_depth+n_qubits*(q_depth-1)
print(tensor_length)

def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RZ_layer(w):
    """Layer of parametrized qubit rotations around the y axis.
    """
    for idx, element in enumerate(w):
        qml.RZ(element, wires=idx)

def entangling_layer(nqubits,weights):
    p = nqubits
    weights_ = (weight for weight in weights)
    for i in range(1,nqubits):
        for j in range(i):
            qml.CNOT(wires=[j,i])
            param = next(weights_)
            # vqc_params.append(param)
            qml.RZ(param, wires=i)
            p+=1
            param = next(weights_)
            # vqc_params.append(param)
            qml.RX(param, wires=i)
            p+=1
            qml.CNOT(wires=[j,i])

dev = qml.device("default.qubit", wires=n_qubits)
@qml.qnode(dev, interface="torch")
def quantum_net(q_input_features, q_weights_flat):
    """
    The variational quantum circuit.
    """
    q_weights = q_weights_flat
    H_layer(n_qubits)
    RZ_layer(q_input_features)
    entangling_layer(nqubits,q_weights)
    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]

    return tuple(exp_vals)
    
class DressedQuantumNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.pre_net = nn.Linear(128, n_qubits)
        self.q_params = nn.Parameter(q_delta * torch.randn(tensor_length))
        self.post_net = nn.Linear(n_qubits, 5)

    def forward(self, input_features):
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = quantum_net(elem, self.q_params).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))

        # return the two-dimensional prediction from the postprocessing layer
        return self.post_net(q_out)

These are the 4 classes that we perform the classification on.

In [ ]:
class_names = [
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
num_classes = len(class_names)
train_images = np.load(f"./saved_data_{dataset}/train_seqs_{seed}.npz")['arr_0']
train_labels = np.load(f"./saved_data_{dataset}/train_labels.npz")['arr_0']
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(train_images[i][0], cmap=plt.cm.binary)
    plt.axis("off")
    plt.title(class_names[train_labels[i]])

We make an array to store all the models loaded from h5 files and other arrays named after the type of noise used which are populated by the performances of the models.

In [ ]:
class Model:
    def __init__(self,path,desc):
        self.path = path
        self.desc = desc
        self.gauss = {'0.01':0.,'0.02':0.,'0.03':0.,'0.04':0.,'0.05':0.,'0.06':0.,'0.07':0.,'0.08':0.,'0.09':0.,'0.1':0.,'0.2':0.,'0.3':0.,'0.4':0.,'0.5':0.,'0.6':0.,'0.7':0.,'0.8':0.,'0.9':0.}
        self.snp = {'0.01':0.,'0.02':0.,'0.03':0.,'0.04':0.,'0.05':0.,'0.06':0.,'0.07':0.,'0.08':0.,'0.09':0.,'0.1':0.,'0.2':0.,'0.3':0.,'0.4':0.,'0.5':0.}
        self.uniform = {'0.01':0.,'0.02':0.,'0.03':0.,'0.04':0.,'0.05':0.,'0.06':0.,'0.07':0.,'0.08':0.,'0.09':0.,'0.1':0.,'0.2':0.,'0.3':0.,'0.4':0.,'0.5':0.,'0.6':0.,'0.7':0.,'0.8':0.,'0.9':0.}
        self.rayleigh ={'0.01':0.,'0.02':0.,'0.03':0.,'0.04':0.,'0.05':0.,'0.06':0.,'0.07':0.,'0.08':0.,'0.09':0.,'0.1':0.,'0.2':0.,'0.3':0.,'0.4':0.,'0.5':0.,'0.6':0.,'0.7':0.,'0.8':0.,'0.9':0.}
        self.perlin = {'1x1':0.,'7x7':0.,'14x14':0.}

    def dump(self):
        np.savez_compressed(file=self.desc,gauss=self.gauss,snp=self.snp,uniform=self.uniform,rayleigh=self.rayleigh,perlin=self.perlin)

In [ ]:
modelnames = {
    "6x2_zz_32_0-Copy1.9413061141967773_0.19680170714855194.h5":"6 qubit 2 layers 94.13 acc 0.1968 loss"
}
# "path_to_file.h5" : "short description of this model"
models = [Model(k,v) for k,v in modelnames.items()]
for model in models:
    print(model.path,model.desc)
    print('-------------------------- '*2)

In [ ]:
test_labels = np.load(f"./saved_data_{dataset}/test_labels.npz")['arr_0']

In [ ]:
def test(input_model,test_x,key):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())
    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1
    print(f"Test accuracy for {key}:", test_acc.numpy())
    return test_acc.numpy()

Here we try to add salt&pepper noise to the test_sequences and then we test our model on them.

In [ ]:
for model in models:
    print(f"Model = {model.desc} at {model.path}")
    obj = torch.load(model.path)
    print("training above model for salt and pepper noise")
    for i in tqdm([i/100 for i in range(1,10)]+[i/10 for i in range(1,6)]):
        model.snp[str(i)] = test(obj,np.load(f"./saved_data_{dataset}/snp_{i}_{seed}.npz")['arr_0'],i)
    print("training above model for gaussian noise")
    for i in tqdm([i/100 for i in range(1,10)]+[i/10 for i in range(1,10)]):
        model.gauss[str(i)] = test(obj,np.load(f"./saved_data_{dataset}/gauss_{i}_{seed}.npz")['arr_0'],i)
    print("training above model for rayleigh noise")
    for i in tqdm([i/100 for i in range(1,10)]+[i/10 for i in range(1,10)]):
        model.rayleigh[str(i)] = test(obj,np.load(f"./saved_data_{dataset}/rayl_{i}_{seed}.npz")['arr_0'],i)
    print("training above model for uniform noise")
    for i in tqdm([i/100 for i in range(1,10)]+[i/10 for i in range(1,10)]):
        model.uniform[str(i)] = test(obj,np.load(f"./saved_data_{dataset}/uniform_0_{i}_{seed}.npz")['arr_0'],i)
    print("training above model for perlin noise")
    for i in tqdm(['1','7','14']):
        model.perlin[str(i)+'x'+str(i)] = test(obj,np.load(f"./saved_data_{dataset}/perlin_{i}_x_{i}_{seed}.npz")['arr_0'],i)
#     model.dump()